In [1]:
!pwd

/content


In [2]:
!ls

booksummaries  booksummaries.tar.gz  sample_data


In [3]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 890kB 15.1MB/s 
     |████████████████████████████████| 3.0MB 21.0MB/s 
     |████████████████████████████████| 1.1MB 36.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ad0ae489d7df817c077621747913668e13d797f2ffccd9e0fec9d15ab70943c5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split
import re

## Data collection and preperation

In [5]:
!wget http://www.cs.cmu.edu/~dbamman/data/booksummaries.tar.gz
!mkdir data/
!tar -xf ./booksummaries.tar.gz

In [6]:
df = pd.read_csv('booksummaries/booksummaries.txt', sep='\t', names=['wiki_id', 'freebase_id', 'title', 'author', 'date', 'genres', 'summary'])
df

,wiki_id,freebase_id,title,author,date,genres,summary
0,620,/m/0hhy,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,/m/0k36,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,/m/0ldx,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,/m/0sww,An Enquiry Concerning Human Understanding,David Hume,NaN,NaN,The argument of the Enquiry proceeds by a ser...
4,2080,/m/0wkt,A Fire Upon the Deep,Vernor Vinge,NaN,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...
...,...,...,...,...,...,...,...
16554,36934824,/m/0m0p0hr,Under Wildwood,Colin Meloy,2012-09-25,NaN,"Prue McKeel, having rescued her brother from ..."
16555,37054020,/m/04f1nbs,Transfer of Power,Vince Flynn,2000-06-01,"{""/m/01jfsb"": ""Thriller"", ""/m/02xlf"": ""Fiction""}",The reader first meets Rapp while he is doing...
16556,37122323,/m/0n5236t,Decoded,Jay-Z,2010-11-16,"{""/m/0xdf"": ""Autobiography""}",The book follows very rough chronological ord...
16557,37132319,/m/0n4bqb1,America Again: Re-becoming The Greatness We Ne...,Stephen Colbert,2012-10-02,NaN,Colbert addresses topics including Wall Stree...


In [7]:
train_test_ratio = 0.9
train_valid_ratio = 7/9
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [8]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    summaries = df['summary'].tolist()
    for summary in summaries:
        summary = str(summary).strip()
        summary = re.sub(r"\s", " ", summary)
        bos_token = '<BOS>'
        eos_token = '<EOS>'
        data += bos_token + ' ' + summary + ' ' + eos_token + '\n'
        
    f.write(data)

In [9]:
build_dataset(df_train, 'data/train.txt')
build_dataset(df_valid, 'data/valid.txt')
build_dataset(df_test, 'data/test.txt')

In [ ]:
N=4
OUTPUT_DIR="model"
TRAIN_FILE="data/train.txt"
VALID_FILE="data/valid.txt"

!CUDA_VISIBLE_DEVICES=$N 
!python run_language_model.py \
  --output_dir=$OUTPUT_DIR \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=$TRAIN_FILE \
  --do_eval \
  --eval_data_file=$VALID_FILE \
  --per_device_train_batch_size=2 \
  --per_device_eval_batch_size=2 \
  --line_by_line \
  --evaluate_during_training \
  --learning_rate 5e-5 \
  --num_train_epochs=5

2020-08-23 08:25:59.454294: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
08/23/2020 08:26:01 - INFO - transformers.training_args -   PyTorch: setting up devices
08/23/2020 08:26:01 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
08/23/2020 08:26:01 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='model', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=True, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Aug23_08-26-01_0e6263499009', logging_first_step=False, logging_steps=500, save_steps=500, save_total_limi

In [11]:
TEST_FILE="data/test.txt"
CUDA_VISIBLE_DEVICES=$N 
!python run_language_modeling.py \
  --output_dir=$OUTPUT_DIR \
  --model_type=gpt2 \
  --model_name_or_path=$OUTPUT_DIR \
  --do_eval \
  --eval_data_file=$TEST_FILE \
  --per_device_eval_batch_size=2 \
  --line_by_line

SyntaxError: ignored

In [ ]:
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=True, return_tensors=”pt”)

In [ ]:
K=k_for_top-k_sampling_decoder
CUDA_VISIBLE_DEVICES=$N 
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path $OUTPUT_DIR \
  --length 300 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k $K \
  --num_return_sequences 5
  --padding_text "This is language modeling, "